In [13]:
!pip install datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [3]:
import sys
import io
import time
import random
import wandb
import pandas as pd
from collections import Counter

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

The main 'geoname' table has the following fields :
---------------------------------------------------
    geonameid         : integer id of record in geonames database
    name              : name of geographical point (utf8) varchar(200)
    asciiname         : name of geographical point in plain ascii characters, varchar(200)
    alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
    latitude          : latitude in decimal degrees (wgs84)
    longitude         : longitude in decimal degrees (wgs84)
    feature class     : see http://www.geonames.org/export/codes.html, char(1)
    feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
    country code      : ISO-3166 2-letter country code, 2 characters
    cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
    admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
    admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80)
    admin3 code       : code for third level administrative division, varchar(20)
    admin4 code       : code for fourth level administrative division, varchar(20)
    population        : bigint (8 byte int)
    elevation         : in meters, integer
    dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
    timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
    modification date : date of last modification in yyyy-MM-dd format

In [4]:
wandb.init(
    project="woher",
    job_type="data-prep"
)

wandb: Currently logged in as: lage. Use `wandb login --relogin` to force relogin


In [6]:
# Raw Dataset
geoname_columns = ["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude", \
                   "feature_class", "feature_code", "country_code", "cc2", "admin1_code", "admin2_code", \
                   "admin3_code", "admin4_code", "population", "elevation", "dem", "timezone", "modification_date"]
df_raw = pd.read_csv("cities500.txt", sep='\t', header=None, names=geoname_columns, index_col=0)
df_raw

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
geonameid,,,,,,,,,,,,,,,,,,
3038999,Soldeu,Soldeu,NaN,42.57688,1.66769,P,PPL,AD,NaN,02,NaN,NaN,NaN,602.0,NaN,1832.0,Europe/Andorra,2017-11-06
3039154,El Tarter,El Tarter,"Ehl Tarter,Эл Тартер",42.57952,1.65362,P,PPL,AD,NaN,02,NaN,NaN,NaN,1052.0,NaN,1721.0,Europe/Andorra,2012-11-03
3039163,Sant Julià de Lòria,Sant Julia de Loria,"San Julia,San Julià,Sant Julia de Loria,Sant J...",42.46372,1.49129,P,PPLA,AD,NaN,06,NaN,NaN,NaN,8022.0,NaN,921.0,Europe/Andorra,2013-11-23
3039604,Pas de la Casa,Pas de la Casa,"Pas de la Kasa,Пас де ла Каса",42.54277,1.73361,P,PPL,AD,NaN,03,NaN,NaN,NaN,2363.0,2050.0,2106.0,Europe/Andorra,2008-06-09
3039678,Ordino,Ordino,"Ordino,ao er di nuo,orudino jiao qu,Ордино,オルデ...",42.55623,1.53319,P,PPLA,AD,NaN,05,NaN,NaN,NaN,3066.0,NaN,1296.0,Europe/Andorra,2018-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8397881,Shuangyang,Shuangyang,"Shuangyang,Shuangyang Xiang,Zhangjiaping,shuan...",31.47660,109.82302,P,PPLA4,CN,NaN,33,NaN,NaN,NaN,0.0,NaN,975.0,Asia/Shanghai,2012-10-08
8398262,Longjing,Longjing,"Longjing,Longjing Xiang,long jing,long jing xi...",31.08943,109.88728,P,PPLA4,CN,NaN,33,NaN,NaN,NaN,0.0,NaN,234.0,Asia/Shanghai,2012-10-08
8398263,Jianping,Jianping,"Jianping,Jianping Xiang,jian ping,jian ping xi...",31.04490,109.92485,P,PPLA4,CN,NaN,33,NaN,NaN,NaN,0.0,NaN,842.0,Asia/Shanghai,2012-10-08


In [7]:
wandb.log({"df_raw": wandb.Table(dataframe=df_raw)})

# Dataset Cleanup

In [8]:
def str_contains(string, test_chars):
    return any(char in test_chars for char in string)

# Remove names containing a character that occurs less than a minimum amount
def remove_least_common_chars(df, name_column, min_char_count=10):
    c = Counter(df[name_column].str.cat())
    total_char_counts = c.most_common()
    too_uncommon_chars = [c[0] for c in total_char_counts if c[1] < min_char_count]
    return df.drop(df.index[df[name_column].apply(lambda x: str_contains(x, too_uncommon_chars)).values]) # Remove remaining names that have a double quote in them


In [9]:
unwanted_columns = ["alternatenames", "feature_class", "feature_code", \
                    "cc2", "admin1_code", "admin2_code", "admin3_code", "admin4_code", \
                    "population", "elevation", "dem", "timezone", "modification_date"]
min_char_count = 25
name_column = "name"

df = df_raw.drop(columns=unwanted_columns)

print("Cleaning up dataset, before starting we have:")
print(f'{len(df)} city names from {len(set(df["country_code"].values))} countries')

# Remove or replace bad stuff
df = df.dropna()
#df.loc[:, name_column] = df[name_column].str.lower() # Only lowercase for now
df.loc[:, name_column] = df[name_column].str.replace(r"\(.*\)","", regex=True) # Remove substrings within parenthesis
df.loc[:, name_column] = df[name_column].str.replace(r"\[.*\]","", regex=True) # Remove substrings within brackets
df.loc[:, name_column] = df[name_column].str.replace(r'\".*\"',"", regex=True) # Remove substrings within quotes
df.loc[:, name_column] = df[name_column].str.replace(r"\'.*\'","", regex=True) # Remove substrings within quotes
df = df.drop(df.index[df[name_column].str.contains('"').values]) # Remove remaining names that have a double quote in them
df[name_column] = df[name_column].map(lambda s: s.split("/")[0]) # If names has alt-name after slash, pick first
df[name_column] = df[name_column].str.strip() # Strip whitespace

df = remove_least_common_chars(df, name_column, min_char_count)


print(f'{len(df)} city names from {len(set(df["country_code"].values))} countries')
print("... is what we ended up with at the end.")

unique_chars = sorted(list(set(df[name_column].str.cat())))
print(f'{len(unique_chars)} unique characters')

df.head(5)


Cleaning up dataset, before starting we have:
30840 city names from 48 countries
30623 city names from 47 countries
... is what we ended up with at the end.
83 unique characters


,name,asciiname,latitude,longitude,country_code
geonameid,,,,,
3038999,Soldeu,Soldeu,42.57688,1.66769,AD
3039154,El Tarter,El Tarter,42.57952,1.65362,AD
3039604,Pas de la Casa,Pas de la Casa,42.54277,1.73361,AD
3039678,Ordino,Ordino,42.55623,1.53319,AD
3040051,les Escaldes,les Escaldes,42.50729,1.53414,AD


In [10]:
wandb.log({"df": wandb.Table(dataframe=df)})

In [25]:
round(12.23454, 5)

12.23454

# Create Llama 2 style prompts

```
<s>[INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model answer </s>
```


In [14]:
from datasets import load_dataset

In [15]:
dataset_name = "mlabonne/guanaco-llama2"
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [19]:
edf = dataset.to_pandas()
display(edf)
edf["text"].values[0]

,text
0,<s>[INST] Can you write a short introduction a...
1,<s>[INST] ¿CUales son las etapas del desarroll...
2,<s>[INST] Can you explain contrastive learning...
3,<s>[INST] I want to start doing astrophotograp...
4,<s>[INST] Método del Perceptrón biclásico: def...
...,...
9841,<s>[INST] What are the risks when you do not c...
9842,<s>[INST] 为什么不能关灯玩手机？ [/INST] 出於各種原因，不建議在黑暗中玩手...
9843,<s>[INST] ¿De qué está hecho el plástico? [/IN...
9844,<s>[INST] Nortzuk ziren Harry Potteren gurasoa...


'<s>[INST] Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research. [/INST] "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to 

In [20]:
PROMPT_TEMPLATE = """<s>[INST]
You are a reverse geocoder for city names.
From a latitude and a longitude, you answer with the most probable city name.
If no real city exists on the location, make up a name that sounds plausible given similar city names in the region.
Extra points for extra stereotypical and clever city names.
Here are the city coordinates: {user_prompt} [/INST] {model_answer} </s>""".replace("\n", " ")

df["latlon"] = df["latitude"].astype(str) + ", " + df["longitude"].astype(str)
llama_df = df.reset_index().apply(lambda x: PROMPT_TEMPLATE.format(user_prompt=x["latlon"], model_answer=x["asciiname"]), axis=1)
llama_df = pd.DataFrame(llama_df,columns=["text"])

llama_df

,text
0,<s>[INST] You are a reverse geocoder for city ...
1,<s>[INST] You are a reverse geocoder for city ...
2,<s>[INST] You are a reverse geocoder for city ...
3,<s>[INST] You are a reverse geocoder for city ...
4,<s>[INST] You are a reverse geocoder for city ...
...,...
30618,<s>[INST] You are a reverse geocoder for city ...
30619,<s>[INST] You are a reverse geocoder for city ...
30620,<s>[INST] You are a reverse geocoder for city ...
30621,<s>[INST] You are a reverse geocoder for city ...


In [21]:
llama_df["text"].values[0]

'<s>[INST] You are a reverse geocoder for city names. From a latitude and a longitude, you answer with the most probable city name. If no real city exists on the location, make up a name that sounds plausible given similar city names in the region. Extra points for extra stereotypical and clever city names. Here are the city coordinates: 42.57688, 1.66769 [/INST] Soldeu </s>'

In [22]:
artifact = wandb.Artifact(name='llama-cities', type='dataset')
artifact.add(wandb.Table(dataframe=llama_df), name="train")
wandb.run.log_artifact(artifact)

<Artifact llama-cities>

In [23]:
wandb.finish()